In [1]:
#Libraries for neo4j
from py2neo import Graph, Node, Relationship

#Libraries for xml
import xml.etree.ElementTree as ET
import pandas as pd
import re

In [2]:
#function to get all the child elements of a given xml element

def get_child_elements(xml_root):
    # Create a list to store all child elements
    elements = []
    #append xml root to elements list
    #elements.append(xml_root)
    # Get all child elements of a given xml element
    for child in xml_root:
        elements.append(child)
        elements.extend(get_child_elements(child))
    return elements

In [3]:
# Set up the Neo4j driver
uri = "neo4j+s://2c9df469.databases.neo4j.io"
user = "neo4j"
password = "qcykrYY8F9afv2Nz1kOKpK8Q0dSNAfL3RJBY1ejvIGY"

# Set up the Neo4j driver
graph = Graph(uri, auth=(user, password))

In [4]:
# Define file to read
#xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Nomina\EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml"
xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Facturas\0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml"
xml_tree = ET.parse(xml_file_path)
xml_root = xml_tree.getroot()

In [5]:
#Read XML Child elements
elements = get_child_elements(xml_root)

#add xml root to read it with their child elements
elements.append(xml_root)

#print child elements
for element in elements:
    print(element.tag, element.attrib)

{http://www.sat.gob.mx/cfd/3}Emisor {'Rfc': 'FIS780810KQ9', 'Nombre': 'FISACERO, S.A.P.I. DE C.V.', 'RegimenFiscal': '601'}
{http://www.sat.gob.mx/cfd/3}Receptor {'Rfc': 'FAN540305I15', 'Nombre': 'GENVAMEX, S.A. DE C.V. AV. NOGALAR SUR No. 301', 'UsoCFDI': 'P01'}
{http://www.sat.gob.mx/cfd/3}Conceptos {}
{http://www.sat.gob.mx/cfd/3}Concepto {'ClaveProdServ': '84111506', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago', 'ValorUnitario': '0', 'Importe': '0'}
{http://www.sat.gob.mx/cfd/3}Complemento {}
{http://www.sat.gob.mx/Pagos}Pagos {'Version': '1.0'}
{http://www.sat.gob.mx/Pagos}Pago {'FechaPago': '2020-07-02T12:00:00', 'FormaDePagoP': '03', 'MonedaP': 'USD', 'TipoCambioP': '23.089300', 'Monto': '21355.97', 'NumOperacion': 'AUT. 70761686', 'CtaOrdenante': '058580012472100218', 'CtaBeneficiario': '112962000030952390'}
{http://www.sat.gob.mx/Pagos}DoctoRelacionado {'IdDocumento': '9FBCCDBB-2B12-4D3D-B872-D01048B95B61', 'Serie': 'IA', 'Folio': '000000481', 'MonedaDR': 'USD'

In [6]:
for element in elements:
    if element.attrib:
        print(re.sub(r"{.*?}", "", element.tag), element.attrib)

Emisor {'Rfc': 'FIS780810KQ9', 'Nombre': 'FISACERO, S.A.P.I. DE C.V.', 'RegimenFiscal': '601'}
Receptor {'Rfc': 'FAN540305I15', 'Nombre': 'GENVAMEX, S.A. DE C.V. AV. NOGALAR SUR No. 301', 'UsoCFDI': 'P01'}
Concepto {'ClaveProdServ': '84111506', 'Cantidad': '1', 'ClaveUnidad': 'ACT', 'Descripcion': 'Pago', 'ValorUnitario': '0', 'Importe': '0'}
Pagos {'Version': '1.0'}
Pago {'FechaPago': '2020-07-02T12:00:00', 'FormaDePagoP': '03', 'MonedaP': 'USD', 'TipoCambioP': '23.089300', 'Monto': '21355.97', 'NumOperacion': 'AUT. 70761686', 'CtaOrdenante': '058580012472100218', 'CtaBeneficiario': '112962000030952390'}
DoctoRelacionado {'IdDocumento': '9FBCCDBB-2B12-4D3D-B872-D01048B95B61', 'Serie': 'IA', 'Folio': '000000481', 'MonedaDR': 'USD', 'MetodoDePagoDR': 'PPD', 'NumParcialidad': '1', 'ImpSaldoAnt': '1017.02', 'ImpPagado': '1017.02', 'ImpSaldoInsoluto': '0.00'}
DoctoRelacionado {'IdDocumento': 'D8CD5D20-6BC0-41A5-913D-AF93F2C173A4', 'Serie': 'IA', 'Folio': '000000483', 'MonedaDR': 'USD', 'Me

In [7]:
#cleantag
for element in elements:
    cleaned_tag = re.sub(r"{.*?}", "", element.tag)
    print(cleaned_tag)

Emisor
Receptor
Conceptos
Concepto
Complemento
Pagos
Pago
DoctoRelacionado
DoctoRelacionado
DoctoRelacionado
TimbreFiscalDigital
Comprobante


In [8]:

# Traverse the XML tree and create nodes and relationships
for parent in xml_root:
    parent_node = Node(parent.tag, **parent.attrib)
    graph.create(parent_node)
    
# Save the nodes and relationships to the graph database
#graph.commit()


testing on row below

In [36]:
# Traverse the XML tree and create nodes and relationships
for parent in xml_root:
    parent_node = Node(parent.tag, **parent.attrib)
    graph.create(parent_node)
    for child in parent:
        child_node = Node(child.tag, **child.attrib)
        graph.create(child_node)
        graph.create(Relationship(parent_node, 'HAS_CHILD', child_node))

In [17]:
# writing code, testing cleaned tag

# Traverse the XML tree and create nodes and relationships
for parent in xml_root:
    # cleaned parent tag
    parent_cleaned_tag = re.sub(r"{.*?}", "", parent.tag)
    # create node
    parent_node = Node(parent_cleaned_tag, **parent.attrib)
    graph.create(parent_node)

    for child in parent:
        # cleaned child tag
        child_cleaned_tag = re.sub(r"{.*?}", "", child.tag)
        # create child node
        child_node = Node(child_cleaned_tag, **child.attrib)
        graph.create(child_node)
        # create relationship
        graph.create(Relationship(parent_node, 'HAS_CHILD', child_node))

# Save the nodes and relationships to the graph database

    
# Save the nodes and relationships to the graph database
#graph.commit()


In [19]:
# Define your Cypher query
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
"""

# Run the query and store the results in a variable
results = graph.run(query)

# Loop through the results and print each record
for record in results:
    print(record)

Node('Conceptos')	HAS_CHILD(Node('Conceptos'), Node('Concepto', Cantidad='1', ClaveProdServ='84111506', ClaveUnidad='ACT', Descripcion='Pago', Importe='0', ValorUnitario='0'))	Node('Concepto', Cantidad='1', ClaveProdServ='84111506', ClaveUnidad='ACT', Descripcion='Pago', Importe='0', ValorUnitario='0')
Node('Complemento')	HAS_CHILD(Node('Complemento'), Node('Pagos', Version='1.0'))	Node('Pagos', Version='1.0')
Node('Complemento')	HAS_CHILD(Node('Complemento'), Node('TimbreFiscalDigital', FechaTimbrado='2020-08-06T10:00:54', NoCertificadoSAT='00001000000404481161', RfcProvCertif='ASE0209252Q1', SelloCFD='KYzHfFCfdi3rSOpAISJPq3K3OVj5QfmL2FVAAW/mU0+vgVext/LkwTXirfefSEeob+eC2O9ODS1/bs7DK0BwEJYRYULBH1ENTlGq19feq+Rv3N5nYDOV7hvQZ+K/jbRg1X4ms87TdUKP8r+dzCzHHWXUIDDl32NqkrG1r897ZVe2n3qjNowP3lnJa2U6kXMQHUcCubxCZR46J8eMW+uWOd6PnjxLpqQfv/0cpCSRmThEN/kIdKG6cIxsH0ZppQSTc9QU61xFRdz7nAp/vpjqBhIJHOZZjLO0U4y8SSRAmABFSMNnWdON44itg9XXwIcvedtVfzrV2uBC4nNAEsRC3w==', SelloSAT='UWWsOgtHZHB/UO0pEvSx4sVgpFrEe3Zy

In [ ]:
# Create a NeoVis object with your database credentials
vis = NeoVis(uri", user, password)

# Define the Cypher query to execute
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
"""

# Render the query results as a graph in the notebook
vis.render(query)

In [16]:
#delete all nodes and relationships
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [ ]:
#reference code IGNORE TO EXECUTE

# Define the Py2neo graph
graph = Graph()

# Traverse the XML tree and create nodes and relationships
for parent in xml_root:
    parent_node = Node(parent.tag, **parent.attrib)
    graph.create(parent_node)
    
    for child in parent:
        child_node = Node(child.tag, **child.attrib)
        graph.create(child_node)
        graph.create(Relationship(parent_node, 'HAS_CHILD', child_node))

# Save the nodes and relationships to the graph database
graph.commit()
